In [1]:
from src.dataset import FoursquareNYC

ds = FoursquareNYC(batch_size=8, num_workers=1)
ds.prepare_data()


Dataset already downloaded!
Dataset already extracted!
Dateset statistics before filtering:
Number of users: 1083, with min = 100, max = 2693, and avg: 209.76731301939057
Number of venues: 38333, with min = 1, max = 1145, and avg: 5.926434142905591
Number of venue categories: 400
Dateset statistics after filtering:
Number of users: 1081, with min = 20, max = 2511, and avg: 136.6096207215541
Number of venues: 5128, with min = 10, max = 1145, and avg: 28.797776911076443
Number of venue categories: 320
Spatial graph sparsity: 0.9934740076810561
Temporal graph sparsity: 0.9182665190772997


In [7]:
import ipywidgets as widgets
import numpy as np
from IPython.display import display
widget = widgets.Output()
# with widget:
#     display(ds.poi_trajectories)
# print(ds.user_train_trajectories.iloc[400])
ds.setup(stage='fit')
ds.setup(stage='test')
dl_train = ds.train_dataloader()
dl_test = ds.test_dataloader()
x, y, lens = next(iter(dl_train)) 
user_ids = x[0]
pois = x[1]
print(user_ids.unsqueeze((1,2)).repeat(1, pois.size(1)).shape)
print(pois.shape)
# print(lens)
# print(x[1].shape)
# print(next(iter(dl_test)))
# for batch in dl_test:
#     pass

# display(widget)

# lengths1 = np.array(ds.user_train_trajectories['Time Slot'].apply(len))
# lengths2 = np.array(ds.user_train_trajectories['Venue ID'].apply(len))
# lengths3 = np.array(ds.user_train_trajectories['Geohash ID'].apply(len))

# ds.plot_distribution(lengths)


TypeError: unsqueeze() takes 1 positional argument but 2 were given